In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import os


In [2]:
data = pd.read_parquet('laion/part-00000-9230b837-b1e0-4254-8b88-ed2976e9cee9-c000.snappy.parquet')

In [3]:
data = data[:100000]

In [4]:
dw = data[data['WIDTH'] > 512]
dw = dw[dw['HEIGHT'] > 512]
# dw = dw[dw['WIDTH'] == dw['HEIGHT']]

dw

,URL,TEXT,WIDTH,HEIGHT,similarity,hash,punsafe,pwatermark,aesthetic
2,https://i0.wp.com/mystylosophy.com/wp-content/...,ChristianDior-Paris-GoldenAge-Dior-DiorGowns-G...,1093.0,729.0,0.320298,-2584362159348276700,0.000044,0.136713,7.591557
9,https://photos.smugmug.com/OP-W-Bloomfield-v-W...,. The Walled Lake Western Warriors defeated th...,1024.0,624.0,0.329358,-5039974171770319940,0.006741,0.238279,7.310762
10,https://p1.liveauctioneers.com/385/116367/5981...,Chinese carved and gilt wood figure of an elder,729.0,1100.0,0.346506,7568100471795618323,0.000043,0.063718,7.313166
11,https://www.jpost.com//HttpHandlers/ShowImage....,Pop legend Boy George and his band are expecte...,822.0,537.0,0.322856,-3174780206027396887,0.048171,0.496625,7.356823
21,https://i.pinimg.com/736x/e4/bc/a9/e4bca97b76c...,Delicious antique French gilt / silver metal f...,720.0,1280.0,0.314220,-7705496269202112015,0.000334,0.110757,7.452411
...,...,...,...,...,...,...,...,...,...
99980,https://i0.wp.com/visit-twincities.com/wp-cont...,Canadian Pacific Holiday Train,1080.0,675.0,0.308310,8840672684899994712,0.000062,0.155659,7.100293
99981,https://i.pinimg.com/originals/58/37/0a/58370a...,Vanessa Hudgens Is Making A Christmas Movie Wi...,582.0,582.0,0.343916,7970081860767275860,0.186657,0.212129,7.642546
99982,https://photos.smugmug.com/Travel/Southeast-Te...,"Texas State Railroad, Rusk / Palestine. New en...",800.0,560.0,0.319057,7743425272809625377,0.000026,0.019720,7.242507
99996,https://lolalambchops.com/wp/wp-content/upload...,"For Finding Dory Party ideas, head to the Doll...",800.0,1024.0,0.377010,-5576514497821840226,0.000333,0.058042,7.036367


In [8]:
test = dw

titles = []
ids = []

for i in range(len(test)):
    print(i)
    # Skip if image exists
    if os.path.isfile('data/real/' + str(test.iloc[i]['hash']) + '.png'):
        print(test.iloc[i]['TEXT'])
        titles.append(test.iloc[i]['TEXT'])
        ids.append(test.iloc[i]['hash'])

        # Show image
        # img = im.imread('data/real/' + str(test.iloc[i]['hash']) + '.png')
        # plt.imshow(img)
        # plt.show()
        continue
      
    try:
      response = requests.get(test.iloc[i]['URL'], timeout=30)
      img = Image.open(BytesIO(response.content))

      if not img.mode == "RGB":
            img = img.convert("RGB")

      # plt.imshow(img)
      # plt.show()

      img = np.array(img).astype(np.uint8)
      crop = min(img.shape[0], img.shape[1])
      h, w, = img.shape[0], img.shape[1]
      img = img[(h - crop) // 2:(h + crop) // 2,
            (w - crop) // 2:(w + crop) // 2]

      image = Image.fromarray(img)
      image = image.resize((512, 512), resample=Image.BILINEAR)

      # image = self.flip(image)
      img = np.array(image).astype(np.uint8)

      # Save image
      try:
        image.save('data/real/' + str(test.iloc[i]['hash']) + '.png')
      except Exception as e:
        print(e)

      print(test.iloc[i]['TEXT'])
      titles.append(test.iloc[i]['TEXT'])
      ids.append(test.iloc[i]['hash'])

      #plt.imshow(img)
      #plt.show()
    except:
      print(test.iloc[i]['TEXT'])
      print('Error')
      continue

# Write titles to file
with open('laion/titles.txt', 'w') as f:
    for item in titles:
        f.write("%s\n" % item)


0
ChristianDior-Paris-GoldenAge-Dior-DiorGowns-Gowns-ParisExhibit
1
. The Walled Lake Western Warriors defeated the West Bloomfield Lakers 19-14 in the Xenith Prep Kickoff Classic game played on Thursday August 24, 2017 at Wayne State University.  (Oakland Press photo by Ken Swart)
2
Chinese carved and gilt wood figure of an elder
3
Pop legend Boy George and his band are expected to arrive for their Israel concert in November 2017.  (photo credit: PR)
Error
4
Delicious antique French gilt / silver metal flaming sacred heart box / reliquary / ex-voto, with the initial M surmounted by a cross, surrounded by foliage motifs . The ex-voto dates from the late 1800's. The box opens so that one can add a thank you message to the Virgin Mary.
5
A number of chocolate muffins with chocolate chips in them sitting on a flipped over baking pan on a marble surface.
6
Engagement session at Dow's Lake in Ottawa with the couple sitting on a bench surrounded by beautiful greenery
7
closeup of lemon short

/var/folders/mk/7wnwqr192b79qr8pf4k24k300000gn/T/ipykernel_64466/3548690974.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  image = image.resize((512, 512), resample=Image.BILINEAR)


Keanu-Reeves-and-Common-in-John-Wick-Chapter-2
354
"""Cheyenne ladies partially beaded moccasins, with beaded edgework, round beaded tongue, and red, white, blue, pink, green, brown, white beading, 10""""L."""
355
1133810 lemon cream cheese bars for social post photo by lutzflcat
356
March bullet journal cover
357
Lemon + Strawberry Layer Spring Cake with roses
358
tuscany castle wedding 86
Error
359
Used 2007 Bentley Continental GTC  For Sale 0 In Greenwich, CT
360
Fish Tacos with Beet-Nectarine Salsa and Baby Greens (gluten free)
361
Low-carb Maple Spiced Granola
362
CTB Kaya & Butter Toast (Double) with Soft Boiled Eggs
363
Red Rock Canyon - Things To Do In Las Vegas
Error
364
Best 25+ Country Dining Tables Ideas On Pinterest
365
Watkins Glen State Park, New York
366
Whimsical Anatomical Collages (UPDATE)
367
Russell Crowe in Les Misérables.
Error
368
In mid-June in the village of Lovozero in the Murmansk Region, right in the heart of Saami culture, a picturesque lakeside location h

In [6]:
contents = os.listdir('data/samples')
contents = ['samples/' + x for x in contents if x.endswith('.png')]

df = pd.DataFrame(contents, columns=['filename'])
df['label'] = 0

contents = os.listdir('data/real')
contents = ['real/' + x for x in contents if x.endswith('.png')]

df_real = pd.DataFrame(contents, columns=['filename'])
df_real['label'] = 1

df = df.append(df_real)

df

/var/folders/mk/7wnwqr192b79qr8pf4k24k300000gn/T/ipykernel_64466/1345939179.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_real)


,filename,label
0,samples/03836.png,0
1,samples/04181.png,0
2,samples/03188.png,0
3,samples/02296.png,0
4,samples/01947.png,0
...,...,...
25220,real/4820093969287475937.png,1
25221,real/4262334731317831201.png,1
25222,real/2011702596060542276.png,1
25223,real/1024904195775613537.png,1


In [7]:
df.to_csv('data/stable_diffusion_detection.csv', index=False)